In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
path = str(Path(Path('File.py').parent.absolute()).parent.absolute())
sys.path.insert(0, path)

path = str(Path(Path('thermo.py').parent.absolute()).parent.absolute())
sys.path.insert(0, path)

import numpy as np
import matplotlib as mlab
import matplotlib.pyplot as plt
import scipy
import xarray as xr
import os as os
import glob
import matplotlib.gridspec as gridspec
from tools.cbook import pickle2Obj, interp_z, compute_thetae
import tools
from tools.FV3_tools import read_solo_fields
from tools.CM1_tools import read_cm1_fields
import netCDF4 as ncdf

from pltbook import nice_clevels, nice_mxmnintvl, container, plot_contour_row

import matplotlib.colors as mcolors

cmap_data = [(1.0, 1.0, 1.0),
         (0.3137255012989044, 0.8156862854957581, 0.8156862854957581),
         (0.0, 1.0, 1.0),
         (0.0, 0.8784313797950745, 0.501960813999176),
         (0.0, 0.7529411911964417, 0.0),
         (0.501960813999176, 0.8784313797950745, 0.0),
         (1.0, 1.0, 0.0),
         (1.0, 0.6274510025978088, 0.0),
         (1.0, 0.0, 0.0),
         (1.0, 0.125490203499794, 0.501960813999176),
         (0.9411764740943909, 0.250980406999588, 1.0),
         (0.501960813999176, 0.125490203499794, 1.0),
         (0.250980406999588, 0.250980406999588, 1.0),
         (0.125490203499794, 0.125490203499794, 0.501960813999176),
         (0.125490203499794, 0.125490203499794, 0.125490203499794),
         (0.501960813999176, 0.501960813999176, 0.501960813999176),
         (0.8784313797950745, 0.8784313797950745, 0.8784313797950745),
         (0.9333333373069763, 0.8313725590705872, 0.7372549176216125),
         (0.8549019694328308, 0.6509804129600525, 0.47058823704719543),
         (0.6274510025978088, 0.42352941632270813, 0.23529411852359772),
         (0.4000000059604645, 0.20000000298023224, 0.0)]

_nthreads = 2

# draw filled contours.
wclevs = [0, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40, 50, 60, 70]

tclevs = list(-40.0 + 2*np.arange(41))

teclevs = list(300. + 5*np.arange(70))

bclevs = list(-0.4+ 0.02*np.arange(100))
pclevs = list(-15.0 + np.arange(31))

dclevs = list(-20.0 + np.arange(41))

zlevels = 25.0 + 50.*np.arange(300)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
def write_forcing(model, filename):

    debug = True
    
    try: ncfile.close()  # just to be safe, make sure dataset is not already open.
    except: pass

    ncfile = ncdf.Dataset(filename,mode='w',format='NETCDF4_CLASSIC') 

    #print(ncfile)

    x_dim = ncfile.createDimension('nx', model['xc'].size)     
    y_dim = ncfile.createDimension('ny', model['yc'].size)   
    z_dim = ncfile.createDimension('nz', model['zc'].shape[1]) 
    t_dim = ncfile.createDimension('time', None)
    
    # for dim in ncfile.dimensions.items():
    #     print(dim)
        
    x = ncfile.createVariable('xh', np.float32, ('nx',))
    x.units = 'meters'
    
    y = ncfile.createVariable('yh', np.float32, ('ny',))
    y.units = 'meters'
    
    z = ncfile.createVariable('zh', np.float32, ('time','nz','ny','nx'))
    z.units = 'meters'
    
    t = ncfile.createVariable('time', np.float32, ('time',))
    t.units = 'sec'
    
    den = ncfile.createVariable('den', np.float32, ('time','nz','ny','nx'))
    den.units = 'kg/m^3'
    
    # write coordinates and data
    
    x[:] = model['xc'][:]
    y[:] = model['yc'][:]
    t[:] = model['sec'][:]
    
    for n in np.arange(model['den'].shape[0]):
    
        den[n,:,:,:] = model['den'][n,:,:,:]
        z[n,:,:,:]   = model['zc'][n,:,:,:]
    
    # print(ncfile)
    ncfile.close()

# if debug, make sure file is written

    if debug:
        print('\n ------DEBUG WRITE FORCING---------\n')
        try: ncfile.close()  # just to be safe, make sure dataset is not already open.
        except: pass
        ncfile = ncdf.Dataset(filename,mode='r') 
        print(ncfile, '\n')
        print('\nGRID DIMENSIONS\n',ncfile.variables['den'].shape)
        print('\nTIME\n',ncfile.variables['time'][:])
        print('\nXH\n',ncfile.variables['xh'][:])
        print('\nYH\n',ncfile.variables['yh'][:])
        print('\nZH\n',ncfile.variables['zh'][0,:,0,0])
        den = ncfile.variables['den']
        print('\nDENSITY COLUMN\n',den[0,:,0,0])
        ncfile.close()


In [30]:
paths = {}
paths['cm1_3km_hdd125'] = '/work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00'
#paths['cm1_2km_hdd125'] = '/work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_test_hdd125_C2000_00'
#paths['cm1_1km_hdd125'] = '/work/wicker/Odin_scr/cm1r20.3/run/bubble_1km_dt01_hdd125_C2000_00'

cm1 = []

for key in paths:
    print("Reading: ",key)
    cm1.append( read_cm1_fields(paths[key], vars = ['den'], file_pattern=None, ret_dbz=False, ret_ds=False) )

Reading:  cm1_3km_hdd125
------------------------------------------------------------------------------------------------------------------------ 

 Added default filename to path input:  /work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00/cm1out.nc
------------------------------------------------------------------------------------------------------------------------ 

 Reading:  /work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00/cm1out.nc 

 Completed reading in:  /work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00/cm1out.nc 

------------------------------------------------------------------------------------------------------------------------


In [37]:
write_forcing(cm1[0], 'cm1_3D.nc')



 ------DEBUG WRITE FORCING---------

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): nx(64), ny(64), nz(60), time(61)
    variables(dimensions): float32 xh(nx), float32 yh(ny), float32 zh(time, nz, ny, nx), float32 time(time), float32 den(time, nz, ny, nx)
    groups:  


GRID DIMENSIONS
 (61, 60, 64, 64)

TIME
 [   0.   60.  120.  180.  240.  300.  360.  420.  480.  540.  600.  660.
  720.  780.  840.  900.  960. 1020. 1080. 1140. 1200. 1260. 1320. 1380.
 1440. 1500. 1560. 1620. 1680. 1740. 1800. 1860. 1920. 1980. 2040. 2100.
 2160. 2220. 2280. 2340. 2400. 2460. 2520. 2580. 2640. 2700. 2760. 2820.
 2880. 2940. 3000. 3060. 3120. 3180. 3240. 3300. 3360. 3420. 3480. 3540.
 3600.]

XH
 [  1500.0001   4500.       7500.0005  10500.001   13500.001   16500.
  19500.      22500.002   25500.002   28500.002   31500.002   34500.
  37500.      40500.004   43500.004   46500.004   49500.004   52500.004
  55500.004   58500.004   61

In [43]:
paths = {}
# paths['solo1'] = '/work/wicker/Odin_scr/solo/bubble_3km_dt20_hdd125_C2000_00'
# paths['solo2'] = '/work/wicker/Odin_scr/solo/bubble_3km_makenh_hdd125_C2000_00'
#paths['solo3'] = '/work/wicker/Odin_scr/solo/bubble_1km_dt06_hdd125_C2000_00'
paths['solo1'] = '/work/wicker/Odin_scr/solo/bubble_3km_TESTA_hdd125_C2000_00'

solo = []
for key in paths:
    print("Reading: ",key)
    solo.append( read_solo_fields(paths[key], vars = ['den'], file_pattern=None, ret_dbz=False, ret_ds=False) )

Reading:  solo1
------------------------------------------------------------------------------------------------------------------------ 

 Added default filename to path input:  /work/wicker/Odin_scr/solo/bubble_3km_TESTA_hdd125_C2000_00/atmos_hifreq.nc
------------------------------------------------------------------------------------------------------------------------ 

 Reading:  /work/wicker/Odin_scr/solo/bubble_3km_TESTA_hdd125_C2000_00/atmos_hifreq.nc 

 Completed reading in:  /work/wicker/Odin_scr/solo/bubble_3km_TESTA_hdd125_C2000_00/atmos_hifreq.nc 

------------------------------------------------------------------------------------------------------------------------


In [42]:
solo[0]['row'][30,:,32,32]

array([-1.1288311 , -1.1253719 , -1.120539  , -1.1141691 , -1.1063995 ,
       -1.0971593 , -1.0864443 , -1.0744374 , -1.060859  , -1.0451914 ,
       -1.0277435 , -1.00907   , -0.9893345 , -0.96847844, -0.94661456,
       -0.92401737, -0.9010039 , -0.87807035, -0.8547573 , -0.83056486,
       -0.806598  , -0.78291047, -0.7591477 , -0.73551875, -0.7119455 ,
       -0.68852216, -0.665241  , -0.64216673, -0.6193183 , -0.5967936 ,
       -0.5745548 , -0.5527081 , -0.5312407 , -0.51018035, -0.48957   ,
       -0.4692163 , -0.4493082 , -0.42980227, -0.4108385 , -0.39197484,
       -0.37100416, -0.3492002 , -0.33246726, -0.32743242, -0.3129764 ,
       -0.28764945, -0.26164895, -0.23866375, -0.21857204, -0.20002493,
       -0.18298677, -0.16699517, -0.15220895, -0.13828771, -0.125431  ,
       -0.11334952, -0.10215463, -0.09133916, -0.08088534, -0.06965762],
      dtype=float32)

In [44]:
solo[0]['row'][30,:,32,32] - solo[0]['den'][30,:,32,32]

array([0.0125556 , 0.01137042, 0.01111555, 0.01041412, 0.00998366,
       0.0095799 , 0.00930238, 0.00906718, 0.01027572, 0.01225722,
       0.0137223 , 0.01521909, 0.01685798, 0.01846278, 0.02007002,
       0.0218913 , 0.02386773, 0.02591342, 0.02750123, 0.02837527,
       0.02949119, 0.03087366, 0.03207046, 0.03329188, 0.03440815,
       0.03549427, 0.03659952, 0.03772998, 0.03885335, 0.0400148 ,
       0.0411939 , 0.0424006 , 0.04366049, 0.0449841 , 0.04636186,
       0.04782483, 0.04938346, 0.05115294, 0.05296561, 0.05447769,
       0.05548176, 0.05599526, 0.0573833 , 0.05914813, 0.0591588 ,
       0.0575303 , 0.05626981, 0.05544758, 0.05539489, 0.05539164,
       0.0557165 , 0.05594254, 0.05638803, 0.05669653, 0.05716329,
       0.05748904, 0.05785722, 0.05790002, 0.05807639, 0.05712472],
      dtype=float32)

In [48]:
write_forcing(solo[0], 'solo_3D.nc')


 ------DEBUG WRITE FORCING---------

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): nx(63), ny(63), nz(60), time(60)
    variables(dimensions): float32 xh(nx), float32 yh(ny), float32 zh(time, nz, ny, nx), float32 time(time), float32 den(time, nz, ny, nx)
    groups:  


GRID DIMENSIONS
 (60, 60, 63, 63)

TIME
 [  60.  120.  180.  240.  300.  360.  420.  480.  540.  600.  660.  720.
  780.  840.  900.  960. 1020. 1080. 1140. 1200. 1260. 1320. 1380. 1440.
 1500. 1560. 1620. 1680. 1740. 1800. 1860. 1920. 1980. 2040. 2100. 2160.
 2220. 2280. 2340. 2400. 2460. 2520. 2580. 2640. 2700. 2760. 2820. 2880.
 2940. 3000. 3060. 3120. 3180. 3240. 3300. 3360. 3420. 3480. 3540. 3600.]

XH
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 6